In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
#import xgboost as xgb

In [5]:
train_data = pd.read_csv('diamonds_train.csv')
test_data = pd.read_csv('diamonds_test.csv')

In [19]:
train_data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,1.21,4,1,4,62.4,58.0,4268,6.83,6.79,4.25
1,0.32,3,3,4,63.0,57.0,505,4.35,4.38,2.75
2,0.71,1,4,5,65.5,55.0,2686,5.62,5.53,3.65
3,0.41,2,7,3,63.8,56.0,738,4.68,4.72,3.00
4,1.02,5,4,3,60.5,59.0,4882,6.55,6.51,3.95


In [7]:
train_data.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

In [60]:
train_data.x.value_counts()

4.37     353
4.34     331
4.38     330
4.33     326
4.32     321
4.35     304
4.39     299
4.36     295
4.40     285
4.42     281
4.31     280
4.41     269
4.30     265
4.29     262
4.28     241
4.43     239
4.74     212
4.44     209
4.46     209
4.75     207
4.77     205
5.72     203
4.45     200
5.73     199
4.79     198
5.74     198
4.78     195
6.39     195
4.27     192
6.38     192
        ... 
10.14      1
10.23      1
9.10       1
9.25       1
9.17       1
9.13       1
3.86       1
9.54       1
8.79       1
9.24       1
8.74       1
9.30       1
9.26       1
9.03       1
8.89       1
10.01      1
9.49       1
9.04       1
9.05       1
9.15       1
9.36       1
9.66       1
8.97       1
9.41       1
3.77       1
9.38       1
9.51       1
9.23       1
3.81       1
9.42       1
Name: x, Length: 543, dtype: int64

In [9]:
cortes = {'Ideal': 5, 'Premium': 4, 'Very Good': 3, 'Good': 2, 'Fair': 1}
colores = {'D': 7, 'E': 6, 'F': 5, 'G': 4, 'H': 3, 'I': 2, 'J': 1}
claridad = {'IF': 8, 'VVS1': 7, 'VVS2': 6, 'VS1': 5, 'VS2': 4, 'SI1': 3, 'SI2': 2, 'I1': 1}

In [10]:
train_data['cut'] = train_data['cut'].apply(lambda x: cortes[x])
test_data['cut'] = test_data['cut'].apply(lambda x: cortes[x])
train_data['color'] = train_data['color'].apply(lambda x: colores[x])
test_data['color'] = test_data['color'].apply(lambda x: colores[x])
train_data['clarity'] = train_data['clarity'].apply(lambda x: claridad[x])
test_data['clarity'] = test_data['clarity'].apply(lambda x: claridad[x])

In [11]:
train_data.clarity.value_counts()

3    9749
4    9248
2    6929
5    6066
6    3799
7    2774
8    1327
1     563
Name: clarity, dtype: int64

In [12]:
X = train_data[['carat', 'cut', 'color', 'clarity', 'depth', 'table']]
Y = train_data['price']

In [13]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [55]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.25, random_state=42)

In [49]:
model_f = RandomForestRegressor(n_estimators=1000)

In [50]:
model_f.fit(trainX, trainY)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [51]:
predict_f = model_f.predict(testX)

In [52]:
model_f.score(testX, testY)

0.9794582828988404

In [53]:
test_df = test_data[['carat', 'cut', 'color', 'clarity', 'depth', 'table']]
test_df = sc.transform(test_df)
final_predictions = model_f.predict(test_df)
test_data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.79,3,5,3,62.7,60.0,5.82,5.89,3.67
1,1,1.20,5,1,5,61.0,57.0,6.81,6.89,4.18
2,2,1.57,4,3,3,62.2,61.0,7.38,7.32,4.57
3,3,0.90,3,5,3,63.8,54.0,6.09,6.13,3.90
4,4,0.50,3,5,5,62.9,58.0,5.05,5.09,3.19


In [54]:
submission = pd.DataFrame({
        "id": test_data["id"],
        "price": final_predictions
    })
submission.to_csv('submission.csv', index=False)